In [2]:
import tensorflow as tf


In [4]:
import keras
import numpy as np
import os
import pickle
import cv2

In [5]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [6]:
#Training generator with augmentation

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False,
        fill_mode='nearest')

In [7]:
# Testing generator

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
BATCH_SIZE = 256
N_CLASSES = 3036
LR = 0.001
N_EPOCHS = 50
IMG_SIZE = 96

In [9]:
cd hiragana/

/home/water/japan/hiragana


In [10]:
train_generator = train_datagen.flow_from_directory(
        'training_data',  # this is the target directory
        target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to 150x150
        batch_size=BATCH_SIZE,
        class_mode='sparse')

Found 13468 images belonging to 71 classes.


In [11]:
validation_generator = test_datagen.flow_from_directory(
        'testing_data',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='sparse')

Found 732 images belonging to 71 classes.


In [12]:
from keras.applications.mobilenet import MobileNet

model = MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=True, classes=N_CLASSES, weights=None)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 97, 97, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 48, 48, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 48, 48, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 48, 48, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 48, 48, 32)        128       
__________

In [14]:
# Training

from keras.callbacks import ModelCheckpoint

model_file = "models_4/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=576840,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=30360,
        callbacks=callbacks_list)

Epoch 1/50
    43/576840 [..............................] - ETA: 3956:58:00 - loss: 0.0396 - acc: 0.9874